# Assembly and analysis of Ficus RAD-seq data

See notebook 1 for demultiplexing raw reads to separate files for each sample. This notebook assembles several datasets from the 180 samples contained in the two libraries sequenced across 4 lanes of Illumina HiSeq 3000. 

In [1]:
# conda install ipyrad -c ipyrad

In [1]:
import ipyrad as ip
import pandas as pd
import os
print('ipyrad v.{}'.format(ip.__version__))

ipyrad v.0.9.50


### The reference genome
A reference *Ficus carica* genome was is available at at [NCBI](https://www.ncbi.nlm.nih.gov/genome/?term=common+fig). It can be downloaded by executing the cell below. If the file is already present in the current directory it will skip downloading.

In [3]:
# The reference genome link
reference = (
    "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/"
    "009/761/775/GCA_009761775.1_UNIPI_FiCari_1.0/"
    "GCA_009761775.1_UNIPI_FiCari_1.0_genomic.fna.gz"
)

# get the file name
gzip_fname = reference.split("/")[-1]
fname = gzip_fname[:-3]

# download and decompress file and name it {name}.fna
if os.path.exists(gzip_fname):
    ! gunzip ./GCA_009761775.1_UNIPI_FiCari_1.0_genomic.fna.gz
    print('decompressing: {}'.format(gzip_fname))

if os.path.exists(fname):
    print('fasta file: {}'.format(fname))
else:
    ## Download the reference genome of F. carica
    print('downloading: {}'.format(gzip_fname))
    ! wget $reference
    print('decompressing: {}'.format(gzip_fname))
    ! gunzip ./GCA_009761775.1_UNIPI_FiCari_1.0_genomic.fna.gz
    print('fasta file: {}'.format(fname))

downloading: GCA_009761775.1_UNIPI_FiCari_1.0_genomic.fna.gz
--2020-04-08 17:46:22--  ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/009/761/775/GCA_009761775.1_UNIPI_FiCari_1.0/GCA_009761775.1_UNIPI_FiCari_1.0_genomic.fna.gz
           => ‘GCA_009761775.1_UNIPI_FiCari_1.0_genomic.fna.gz’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /genomes/all/GCA/009/761/775/GCA_009761775.1_UNIPI_FiCari_1.0 ... done.
==> SIZE GCA_009761775.1_UNIPI_FiCari_1.0_genomic.fna.gz ... 103523264
==> PASV ... done.    ==> RETR GCA_009761775.1_UNIPI_FiCari_1.0_genomic.fna.gz ... done.
Length: 103523264 (99M) (unauthoritative)

GCA_009761775.1_UNI 100%[===================>]  98.73M  27.8MB/s    in 3.6s    

2020-04-08 17:46:26 (27.8 MB/s) - ‘GCA_009761775.1_UNIPI_FiCari_1.0_genomic.fna.gz’ 

### Load Demultiplexed fastq files 

See notebook1 for details of demultiplexing. Here we load the four lanes of data as sorted fastq data since this is the way that others would download the data from NCBI. 

In [ ]:
# setup assembly object to load the four libraries
demux1 = ip.Assembly("demux1")
demux1.params.project_dir = "./analysis-ipyrad"
demux1.params.sorted_fastq_path = "./demux_fastqs/lib1lane1_fastqs/*.gz"

demux2 = ip.Assembly("demux2")
demux2.params.project_dir = "./analysis-ipyrad"
demux2.params.sorted_fastq_path = "./demux_fastqs/lib1lane2_fastqs/*.gz"

demux3 = ip.Assembly("demux3")
demux3.params.project_dir = "./analysis-ipyrad"
demux3.params.sorted_fastq_path = "./demux_fastqs/lib2lane1_fastqs/*.gz"

demux4 = ip.Assembly("demux4")
demux4.params.project_dir = "./analysis-ipyrad"
demux4.params.sorted_fastq_path = "./demux_fastqs/lib2lane2_fastqs/*.gz"

# load fastq files for the four lanes of data
demux1.run("1", auto=True, quiet=True, force=True)
demux2.run("1", auto=True, quiet=True, force=True)
demux3.run("1", auto=True, quiet=True, force=True)
demux4.run("1", auto=True, quiet=True, force=True)

In [ ]:
# merge assemblies into a single one
data = ip.merge("allmerged", [demux1, demux2, demux3, demux4])

In [ ]:
# samples that we may want to exclude later
data.stats[data.stats.reads_raw < 100000]

In [ ]:
# drop low coverage samples and the CONTROL 
keep = [i for i in data.samples if i != "FGXCONTROL"]
data = data.branch("ficus-190", keep, force=True)

### Set parameters for assembly

In [ ]:
# set several non-default parameters
data.params.project_dir = "analysis-ipyrad"
data.params.assembly_method = "reference"
data.params.reference_sequence = "./GCA_009761775.1_UNIPI_FiCari_1.0_genomic.fna"
data.params.filter_adapters = 3
data.params.phred_Qscore_offset = 43
data.params.max_Hs_consens = 0.1
data.params.max_shared_Hs_locus = 0.1
data.params.filter_min_trim_len = 50
data.params.output_formats = "lp"

# print parameters for prosperity's sake
data.params

### A summary of the number of reads per Sample.


In [9]:
print("summary of raw read covereage")
print(data.stats.reads_raw.describe().astype(int))

summary of raw read covereage
count         190
mean      6039582
std       8967873
min          5463
25%        250209
50%       2013696
75%       8209965
max      51911404
Name: reads_raw, dtype: int64


### Filtering options explained

From looking closely at the data it appears there are some poor quality reads with adapter contamination, and also that there are some conspicuous long strings of poly repeats, which are probably due to the library being put on the sequencer in the wrong concentration (the facility failed to do a qPCR quantification). Setting the filter parameter in ipyrad to strict (2) uses 'cutadapt' to filter the reads. By default ipyrad would look just for the Illumina universal adapter, but I'm also adding a few additional poly-{A,C,G,T} sequences to be trimmed. These appeared to be somewhat common in the raw data, followed by nonsense.


### Within-sample assembly

Steps 2-5 of ipyrad function to filter and cluster reads, and to call consensus haplotypes within samples. We'll look more closely at the stats for each step after it's finished.


In [11]:
data = ip.load_json("./analysis-ipyrad/ficus-190.json")
pbdata = data.branch("ficus-190-pb")
pbdata.params.reference_sequence = "./GCA_009761775.1_UNIPI_FiCari_1.0_genomic.fna"

loading Assembly: ficus-190
from saved path: ~/Documents/Ficus/analysis-ipyrad/ficus-190.json


In [16]:
pbdata.ipcluster["cores"] = 28
pbdata.run("345", auto=True, force=True)

Parallel connection | 1efffc7603ac: 28 cores
[####################] 100% 0:05:21 | indexing reference   | s3 |
[####################] 100% 0:09:30 | dereplicating        | s3 |
[####################] 100% 0:15:51 | mapping reads        | s3 |
[####################] 100% 0:09:44 | building clusters    | s3 |
[####################] 100% 0:00:34 | calc cluster stats   | s3 |
[####################] 100% 0:09:51 | inferring [H, E]     | s4 |
[####################] 100% 0:00:27 | calculating depths   | s5 |
[####################] 100% 0:00:32 | chunking clusters    | s5 |
[####################] 100% 2:35:15 | consens calling      | s5 |
[####################] 100% 0:01:42 | indexing alleles     | s5 |


In [19]:
# get all samples with <5000 consensus reads
drop = pbdata.stats[pbdata.stats.reads_consens < 1000]
drop.reads_consens

A29_popenoei              570
A62_turbinata             812
A63_turbinata             533
C02_citrifolia             23
C09_costaricana            75
C32_triangleXtrigonata    694
C33_triangle              685
C34_triangle              129
C52_citrifolia            108
Fbul-2                    117
Fcol-1                    199
Fcos-3                    421
Fcot-3                    207
Fgom-2                    481
Fobt-1                    787
Fobt-5                    732
Fpop-1                    396
Fpop-2                    395
Ftria-1                   175
Name: reads_consens, dtype: int64

In [20]:
# get all samples with <5000 consensus reads
keep = pbdata.stats[pbdata.stats.reads_consens >= 1000].index.tolist()

# create branch with only hidepth samples
subdata = pbdata.branch("ficus-190-pb-1Kmin", subsamples=keep)

In [22]:
subdata.run('6', auto=True, force=True)

Parallel connection | 1efffc7603ac: 28 cores
[####################] 100% 0:00:39 | concatenating bams   | s6 |
[####################] 100% 0:00:19 | fetching regions     | s6 |
[####################] 100% 0:00:54 | building database    | s6 |


In [23]:
ficus_min4 = subdata.branch("ficus-190-pb-min4")
ficus_min4.params.min_samples_locus = 4
ficus_min4.params.max_SNPs_locus = 0.25
ficus_min4.params.output_formats = "pslu"
ficus_min4.run("7", auto=True, force=True)

Parallel connection | 1efffc7603ac: 28 cores
[####################] 100% 0:01:00 | applying filters     | s7 |
[####################] 100% 0:15:38 | building arrays      | s7 |
[####################] 100% 0:05:43 | writing conversions  | s7 |
